In [1]:
import sys 
sys.path.append('..')

from util_yours import split_knn_patches
from util_yours import center_split_masking

import numpy as np
import torch
from torch.utils.data import DataLoader 
from data_yours import ALS_50K

In [18]:
ds = ALS_50K(num_points=2048)
dl = DataLoader(ds, batch_size=4, shuffle=False, num_workers=0)
batch = next(iter(dl))
batch = batch[0]
batch = batch.permute(0, 2, 1).contiguous()

In [4]:
output_split_knn = split_knn_patches(batch, mask_ratio=0.7, nsample=16, random=True)

RuntimeError: false INTERNAL ASSERT FAILED at "/share/castor/home/e2405193/MAE3D/pointnet2_ops_lib/pointnet2_ops/_ext-src/src/sampling.cpp":83, please report a bug to PyTorch. CPU not supported

In [4]:
for i in range(len(output_split_knn)):
    print(f"output_split_knn[{i}].shape: {output_split_knn[i].shape}")

NameError: name 'output_split_knn' is not defined

In [5]:
output_center_split_masking = center_split_masking(batch.cuda(), masking_ratio=0.7, patch_size=16)

RuntimeError: selected index k out of range

In [24]:
for i in range(len(output_split_knn)):
    print(f"output_split_knn[{i}].shape: {output_split_knn[i].shape}")

output_split_knn[0].shape: torch.Size([8, 0, 2, 2048])
output_split_knn[1].shape: torch.Size([8, 1, 2, 2048])
output_split_knn[2].shape: torch.Size([8, 0, 2048])
output_split_knn[3].shape: torch.Size([8, 1, 2048])
output_split_knn[4].shape: torch.Size([0])
output_split_knn[5].shape: torch.Size([1])
output_split_knn[6].shape: torch.Size([1])


In [5]:
# import MAE3D model 
from model_yours import MAE3D

In [6]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="../config/experiments"):
    cfg = compose(config_name="DEBUG")


In [7]:
cfg.model

{'patch_size': 16, 'mask_ratio': 0.3, 'num_points': 2048, 'masking_strategy': 'xysplit', 'encoder_dims': 1024, 'decoder_dims': 1024, 'mlp_dim': 2048, 'depth': 6, 'num_heads': 8, 'dim_per_head': 64, 'dropout': 0.0, 'loss_alpha_center': 0.1}

In [8]:
model = MAE3D(cfg.model) 
model = model.cuda()

Number of points: 2048, Patch size: 16, Mask ratio: 0.3, Vis patch num: 90, Mask patch num: 38, 


In [9]:
model = model.cuda()  # Ensure model is on CUDA
xyz = batch.permute(0, 2, 1).contiguous().cuda()


with torch.no_grad():
    output = model(xyz)

Input shape before permute: torch.Size([8, 2048, 3])
Input shape: torch.Size([8, 3, 2048])


RuntimeError: selected index k out of range

In [3]:
B, N, C = xyz.shape
B, N, C


(8, 2048, 3)

In [16]:
from pointnet2_ops import pointnet2_utils

In [15]:
def index_points(points, idx):
    device = points.device
    idx = idx.to(device)
    B = points.shape[0]
    view_shape = list(idx.shape)
    view_shape[1:] = [1] * (len(view_shape) - 1)
    repeat_shape = list(idx.shape)
    repeat_shape[0] = 1
    batch_indices = torch.arange(B, dtype=torch.long).to(device).view(view_shape).repeat(repeat_shape)
    new_points = points[batch_indices, idx, :]
    return new_points

In [13]:
def knn_point(nsample, xyz, new_xyz):
    sqrdists = square_distance(new_xyz, xyz)
    _, group_idx = torch.topk(sqrdists, nsample, dim=-1, largest=False, sorted=False)
    return group_idx

In [14]:
def square_distance(src, dst):
    B, N, _ = src.shape
    _, M, _ = dst.shape
    dist = -2 * torch.matmul(src, dst.permute(0, 2, 1))
    dist += torch.sum(src ** 2, -1).view(B, N, 1)
    dist += torch.sum(dst ** 2, -1).view(B, 1, M)
    return dist

In [19]:
batch = batch.permute(0, 2, 1).contiguous()

In [36]:
masking_ratio = cfg.model.mask_ratio
patch_size = cfg.model.patch_size

B, N, C = batch.shape 

# calculate number of patches
num_patches = N // patch_size # assuming N is num_points
num_masked_patches = int(num_patches * masking_ratio) 
num_vis_patches = num_patches - num_masked_patches

vis_pos = []
mask_pos = [] 
mask_center_pos = [] 
vis_center_pos = [] 
# generate patch_idx and shuffle_idx later 

for i in range(B): 
    pc = batch[i]

    # generate random T 
    if np.random.choice([True, False]): 
        t, axis = torch.median(pc[:,0]), 'x'
    else: 
        t, axis = torch.median(pc[:, 1]), 'y'
    
    # subset the point cloud
    if axis == 'x':
        pc_vis = pc[pc[:, 0] > t] 
        pc_masked = pc[pc[:, 0] <= t]
    elif axis == 'y': 
        pc_vis = pc[pc[:, 1] <= t] 
        pc_masked = pc[pc[:, 1] > t]

    # find patch centers in the subsets 
    vis_centers = pointnet2_utils.furthest_point_sample(pc_vis.cuda().unsqueeze(0), num_vis_patches).cpu().squeeze(0)
    masked_centers = pointnet2_utils.furthest_point_sample(pc_masked.cuda().unsqueeze(0), num_masked_patches).cpu().squeeze(0)

    # select center points 
    vis_center_points = pc_vis[vis_centers]
    masked_center_points = pc_masked[masked_centers]

    # get patch idxs
    vis_patch_idxs = get_kNN_patch_idxs(pc_vis, vis_center_points, k=patch_size)
    masked_patch_idxs = get_kNN_patch_idxs(pc_masked, masked_center_points, k=patch_size)

    # get patches 
    vis_patches = [pc_vis[vis_patch_idxs[:, i]] for i in range(vis_patch_idxs.shape[1])]
    masked_patches = [pc_masked[masked_patch_idxs[:, i]] for i in range(masked_patch_idxs.shape[1])]

    vis_patches_tensor = torch.stack(vis_patches, dim=0)
    masked_patches_tensor = torch.stack(masked_patches, dim=0)

    vis_pos.append(vis_patches_tensor)
    vis_center_pos.append(vis_center_points)
    mask_pos.append(masked_patches_tensor)
    mask_center_pos.append(masked_center_points)

vis_pos = torch.stack(vis_pos, dim=0)
vis_center_pos = torch.stack(vis_center_pos, dim=0)
mask_pos = torch.stack(mask_pos, dim=0)
mask_center_pos = torch.stack(mask_center_pos, dim=0)

idx_all = torch.rand(num_patches).argsort()
vis_patch_idx = idx_all[:num_vis_patches]
mask_patch_idx = idx_all[num_vis_patches:]

shuffle_idx = torch.cat((vis_patch_idx, mask_patch_idx), dim=0)

In [37]:
batch_idx = torch.arange(B, device=batch.device).unsqueeze(-1)
batch_idx.shape


torch.Size([4, 1])

In [38]:
center_pos = torch.cat((vis_center_pos, mask_center_pos), dim=1)

In [39]:
center_pos.shape

torch.Size([4, 128, 3])

In [40]:

center_pos_reshuffle = torch.empty_like(center_pos, device=center_pos.device)
center_pos_reshuffle[batch_idx, shuffle_idx] = center_pos
center_pos = center_pos_reshuffle
point_pos = torch.cat((vis_pos, mask_pos), dim=1)
point_pos_reshuffle = torch.empty_like(point_pos, device=point_pos.device)
point_pos_reshuffle[batch_idx, shuffle_idx] = point_pos
point_pos = point_pos_reshuffle

In [41]:
mask_pos = point_pos[batch_idx, mask_patch_idx]
vis_pos = point_pos[batch_idx, vis_patch_idx]
vis_pos_input = vis_pos.view(B, num_vis_patches * cfg.model.patch_size, 3)


In [42]:
vis_pos_input.shape

torch.Size([4, 1440, 3])

In [45]:
model = model.cpu()

In [47]:
x_pos_embed_encoder = model.pos_embed_encoder(center_pos)
x_pos_embed_decoder = model.pos_embed_decoder(center_pos)
x_pos_embed_encoder = x_pos_embed_encoder[batch_idx, vis_patch_idx].view(B, -1, cfg.model.encoder_dims)
x_pos_embed_encoder = torch.cat((model.cls_token_encoder.repeat(B, 1, 1), x_pos_embed_encoder), dim=1)
x_pos_embed_decoder = x_pos_embed_decoder.view(B, -1, cfg.model.decoder_dims)
x_pos_embed_decoder = torch.cat((model.cls_token_decoder.repeat(B, 1, 1), x_pos_embed_decoder), dim=1)

In [48]:
x_pos_embed_encoder.shape, x_pos_embed_decoder.shape

(torch.Size([4, 91, 1024]), torch.Size([4, 129, 1024]))

In [53]:
print(
    f'vis_pos_input.devie: {vis_pos_input.device}, '
    f'x_pos_embed_encoder.device: {x_pos_embed_encoder.device}, '
    f'x_vis.device: {x_pos_embed_encoder.device}'
)

vis_pos_input.devie: cpu, x_pos_embed_encoder.device: cpu, x_vis.device: cpu


In [54]:
model = model.cpu()
x_vis = model.encoder(vis_pos_input, x_pos_embed_encoder)
x_vis = model.encoder_to_decoder(x_vis)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:

x_full_init = model.token_init[None, None, :].repeat(B, num_patches, 1)
x_mask_init = x_full_init[batch_idx, mask_patch_idx]

x_full = torch.cat((x_vis, x_mask_init), dim=1)
x_full_reshuffle = torch.empty_like(x_full, device=x_full.device)
x_full_reshuffle[batch_idx, shuffle_idx] = x_full

x_full = model.decoder(x_full_reshuffle, x_pos_embed_decoder)